In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import os

In [2]:
#フィールド情報dict_fieldから次に消えるぷよの座標list_show_allを出力
#dict_fieldはkeyとして(x, y) (1≦x≦6, 1≦y≦12)、valueとしてぷよの色を表す数値(色ぷよ：0~4, おじゃまぷよ：5、ぷよなし：6)をもつ計72個の辞書
def rensa_arrow(dict_field):
    
    #i:列, j:段
    
    #初期化
    #list_sumi：確認済みの座標
    list_sumi = []
    #list_show_all:最終的な出力
    list_show_all = []
    #puyo_num:連結数
    puyo_num = 1

    for i in range(1, 7):
        for j in range(1, 13):

            if [i, j] not in list_sumi:
                #確認する座標の初期値設定
                list_same = [(i, j)]  
                list_show = []
                
                #list_sameには次に確認すべき座標を全て格納、一つずつ見ていって要素がなくなったら終了
                while len(list_same) != 0:
                    
                    #list_sameの[0]を確認
                    i_tmp = list_same[0][0]
                    j_tmp = list_same[0][1]
                    
                    #色ぷよの場合
                    if dict_field[(i_tmp, j_tmp)] < 5:
                        #出力候補としてlist_showに格納
                        list_show.append((i_tmp, j_tmp))
                        #次に確認するリストlist_next(上下左右)を作成
                        if (1<i_tmp<6):
                            if (j_tmp==1):
                                list_next = [(i_tmp-1, j_tmp), (i_tmp+1, j_tmp), (i_tmp, j_tmp+1)]
                            elif (j_tmp==12):
                                list_next = [(i_tmp-1, j_tmp), (i_tmp+1, j_tmp), (i_tmp, j_tmp-1)]
                            else:
                                list_next = [(i_tmp-1, j_tmp), (i_tmp+1, j_tmp), (i_tmp, j_tmp-1), (i_tmp, j_tmp+1)]
                        elif (i_tmp==1):
                            if (j_tmp==1):
                                list_next = [(i_tmp+1, j_tmp), (i_tmp, j_tmp+1)]
                            elif (j_tmp==12):
                                list_next = [(i_tmp+1, j_tmp), (i_tmp, j_tmp-1)]
                            else:
                                list_next = [(i_tmp+1, j_tmp), (i_tmp, j_tmp+1), (i_tmp, j_tmp-1)]
                        else:
                            if (j_tmp==1):
                                list_next = [(i_tmp-1, j_tmp), (i_tmp, j_tmp+1)]
                            elif (j_tmp==12):
                                list_next = [(i_tmp-1, j_tmp), (i_tmp, j_tmp-1)]
                            else:
                                list_next = [(i_tmp-1, j_tmp), (i_tmp, j_tmp-1), (i_tmp, j_tmp+1)]

                        #list_nextを確認し、色が同じならlist_same、list_showに格納し、連結数を+1
                        #確認し終わったらlist_sameをlist_sumiに格納

                        for k in list_next:
                            if (dict_field[(i_tmp, j_tmp)]==dict_field[k]) & (k not in list_sumi):
                                list_same.append(k)
                                list_show.append(k)
                                puyo_num += 1
                        list_sumi += list_same

                        #list_same[0]を削除
                        list_same.remove(list_same[0])

                    #おじゃまぷよ、ぷよがない場合はlist_sumiに入れる
                    else:
                        list_sumi.append((i, j))
                        #list_same[0]を削除
                        list_same.remove(list_same[0])  
            
                #連結数が4以上なら次に消えるとしてlist_show_allに格納
                if puyo_num >= 4:
                    list_show_all += list_show
                #初期化
                puyo_num = 1

    list_show_all = list(set(list_show_all))
    
    return(list_show_all)

#list_show_allから消えるぷよの重心を計算し、画像上の座標を出力
def calculate_arrow_point(list_show_all):
    x = 0
    y = 0

    for i in list_show_all:
        x += i[0]
        y += i[1]

    if len(list_show_all) != 0:
        x = x/len(list_show_all)
        y = y/len(list_show_all)
        x_field = int(round((x*(454-43)/5) + (43-(454-43)/5), 0))
        y_field = int(round((y*(43-905)/11) + (905-(43-905)/11), 0))
    else:
        x_field = "error"
        y_field = "error"


    return(x_field, y_field)